# Project: Analysis of WeRateDogs Tweets

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#packages">Import of Packages</a></li>
<li><a href="#gather">Gather Data</a></li>
<li><a href="#asses">Asses Data</a></li>
<li><a href="#clean">Clean Data</a></li>
<li><a href="#analysis">Data Analysis</a></li>
<li><a href="#conc">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The goal of this analysis is to identify factors that can help to predict whether patients show up to their appointments or not. Basis for the analysis is a data set provided by __[kaggle](https://www.kaggle.com/joniarroba/noshowappointments)__ which collects collects

## Import of requiered Packages

In [212]:
import pandas as pd
import numpy as np
import datetime
import requests
import os
#import tweepy

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


<a id='gather'></a>
## Gather Data

### Tweet Archive

In [213]:
# Read Twitter Archive from provided csv file
archive = pd.read_csv('./Twitter_Files/twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [214]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Tweet Image Prediction

In [215]:
# Download Tweet Image Predictions file from udacity server
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

# Save file on local drive
with open (os.path.join('Twitter_Files', url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

# Read file into data frame
image = pd.read_csv('./Twitter_Files/image-predictions.tsv', sep='\t') # set \t as seperator, since it is a .tsv file
image.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [216]:
image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [217]:
image['p1_dog'].value_counts()

True     1532
False    543 
Name: p1_dog, dtype: int64

### Tweet Retweet Count

In [218]:
''' 
my account details
https://developer.twitter.com/en/account/get-started

tokens can be found under Apps - BS_Udacity_Dogs
Consumer API keys
V4mljUBU6tUgbzXo8XORLQAgj (API key)
uVb3YtSQLJwSakJelPOJ8JWzC4zdP2CHlrhyIyow8oAAQXoBF7 (API secret key)

Access token & access token secret
1095995458781106176-ClvYF8zgxLd18YK5lN1c0rrc0MXbXh (Access token)
jwhfSN2VIDgk3mim61Bvc7YKQ1YzhHKpcDMfZ86DYdlwi (Access token secret)
'''

'''
#Tweepy from another user (with error)
api = tweepy.API(auth)                                            #Store data on tweet_json.txt
with open('tweet_json.txt', 'w') as outfile: 
for tweet_id in twitter_archive['tweet_id']:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        #Convert to JSON and write
        json.dump(tweet._json, outfile)
        outfile.write("\n")

    except:  
        print('ERROR: Error on tweet id {}'.format(tweet_id))
        e = os.sys.exc_info()[0]                                  #Save only certain tweet elements in dataframe
elements_to_save = ['id', 'favorite_count', 'retweet_count']
#Convert list to dataframe
data = []

with open('tweet_json.txt', 'r') as readfile:
tweet_json = readfile.readline()

# Read line by line into DataFrame
while tweet_json:
    tweet_dict = json.loads(tweet_json)
    # Create a smaller dict
    data_row = dict((k, tweet_dict[k]) for k in elements_to_save)
    data.append(data_row)

    # Read in JSON line and convert to dict
    tweet_json = readfile.readline()
    df_tweet_info = pd.DataFrame.from_dict(data)

# also check working solution 
https://knowledge.udacity.com/questions/30296

'''

'\n#Tweepy from another user (with error)\napi = tweepy.API(auth)                                            #Store data on tweet_json.txt\nwith open(\'tweet_json.txt\', \'w\') as outfile: \nfor tweet_id in twitter_archive[\'tweet_id\']:\n    try:\n        tweet = api.get_status(tweet_id, tweet_mode=\'extended\')\n        #Convert to JSON and write\n        json.dump(tweet._json, outfile)\n        outfile.write("\n")\n\n    except:  \n        print(\'ERROR: Error on tweet id {}\'.format(tweet_id))\n        e = os.sys.exc_info()[0]                                  #Save only certain tweet elements in dataframe\nelements_to_save = [\'id\', \'favorite_count\', \'retweet_count\']\n#Convert list to dataframe\ndata = []\n\nwith open(\'tweet_json.txt\', \'r\') as readfile:\ntweet_json = readfile.readline()\n\n# Read line by line into DataFrame\nwhile tweet_json:\n    tweet_dict = json.loads(tweet_json)\n    # Create a smaller dict\n    data_row = dict((k, tweet_dict[k]) for k in elements_to_s

<a id='assess'></a>
## Assess Data

### Tweet Archive

In [219]:
# Get a basic idea of columns in table and their values
archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1848,675798442703122432,NaN,NaN,2015-12-12 22:04:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bernie. He just touched a boob for the first time. 10/10 https://t.co/whQKMygnK6,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675798442703122432/photo/1,10,10,Bernie,None,None,None,None
80,877316821321428993,NaN,NaN,2017-06-21 00:06:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Dante. At first he wasn't a fan of his new raincoat, then he saw his reflection. H*ckin handsome. 13/10 for water resistant good boy https://t.co/SHRTIo5pxc",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/877316821321428993/photo/1,https://twitter.com/dog_rates/status/877316821321428993/photo/1",13,10,Dante,None,None,None,None
1323,706265994973601792,NaN,NaN,2016-03-05 23:51:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Kara. She's been trying to solve that thing for 3 days. ""I don't have thumbs,"" she said. 11/10 solid effort https://t.co/lA6a8GefrV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/706265994973601792/photo/1,11,10,Kara,None,None,None,None
71,878776093423087618,NaN,NaN,2017-06-25 00:45:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Snoopy. He's a proud #PrideMonthPuppo. Impeccable handwriting for not having thumbs. 13/10 would love back #PrideMonth https://t.co/lNZwgNO4gS,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/878776093423087618/photo/1,https://twitter.com/dog_rates/status/878776093423087618/photo/1",13,10,Snoopy,None,None,None,puppo
2112,670435821946826752,NaN,NaN,2015-11-28 02:55:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Two unbelievably athletic dogs here. Great form. Perfect execution. 10/10 for both https://t.co/sQuKwSKtDE,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670435821946826752/photo/1,10,10,None,None,None,None,None


In [220]:
# Get data types of columns and number of non-null entries
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

- There a some columns with null values. However, all columns of interest have 2356 non-null values.
- Colummn **timestamp** should be converted to DateTime object to be better suited for analysis

In [221]:
#Get number of duplicated rows
sum(archive.duplicated())

0

In [222]:
# Check for duplicated values in 'tweet_id'
sum(archive.tweet_id.duplicated())   # could have also been analyzed by archive.tweet_id.nunique()  

0

There are no duplicated rows and only unique tweet_ids in this table.

In [223]:
archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


**rating_numerator** is supposed to be around/greater than 10. It ranges between 0 and 1776 and is further analyzed (e.g. for outliers. **rating_denominator** is supposed to be around 10. It ranges between 0 and 170 and is further analyzed (e.g. for outliers.

In [224]:
archive.rating_denominator.value_counts()

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

In [225]:
archive.query('rating_denominator!=10').count()[0]

23

23 rows out of 2356 have a denominator which is not 10 (less than 1% of all rows). These rows will be deleted to ease later analysis.

In [226]:
#create mask with denominators of 10, sicne all other rows will be removed later anyways
archive_10 = archive.query('rating_denominator==10') 
archive_10['rating_numerator'].value_counts().sort_index()

0       2  
1       8  
2       9  
3       19 
4       15 
5       37 
6       32 
7       54 
8       102
9       156
10      461
11      463
12      558
13      351
14      54 
15      2  
17      1  
26      1  
27      1  
75      2  
182     1  
420     2  
666     1  
1776    1  
Name: rating_numerator, dtype: int64

In [227]:
pd.set_option('display.max_colwidth', -1)
archive_10.query('rating_numerator>17').text

188     @dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research                                                                           
189     @s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10                            
290     @markhoppus 182/10                                                                                                                                               
340     RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…                     
695     This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS                                 
763     This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back http

After checking some of the tweets with rating higher than 17, it seems that there are few float ratings where the decimal places are stored in column **rating_numerator** (9.75, 11.27 and 11.26). Those rows will be removed. Ratings greater than or equal to 182 are considered as outliers and will be removed as well (this applies to 5 ratings, two relating to Snoop Dogg and one to Atticus (dog of independence) with a rating of 1776).

In [228]:
# Check entries of name column
archive.name.value_counts()

None        745
a           55 
Charlie     12 
Oliver      11 
Cooper      11 
Lucy        11 
Tucker      10 
Penny       10 
Lola        10 
Bo          9  
Winston     9  
Sadie       8  
the         8  
Toby        7  
Buddy       7  
Bailey      7  
Daisy       7  
an          7  
Bella       6  
Dave        6  
Jack        6  
Jax         6  
Milo        6  
Leo         6  
Stanley     6  
Oscar       6  
Koda        6  
Scout       6  
Rusty       6  
George      5  
           ..  
Barney      1  
Sundance    1  
Bradlay     1  
Brutus      1  
Travis      1  
old         1  
Ralf        1  
Dex         1  
Moofasa     1  
Coleman     1  
Damon       1  
Nugget      1  
Oreo        1  
Lance       1  
Margo       1  
Jarod       1  
Maks        1  
Tebow       1  
Eazy        1  
Napolean    1  
Georgie     1  
Fynn        1  
Beebop      1  
Gòrdón      1  
Willem      1  
Geno        1  
Cuddles     1  
Gilbert     1  
Crouton     1  
Kanu        1  
Name: name, Length: 957,

There are some entries in **name** column which were not parsed correctly, e.g. 'a', 'an', 'such', 'quite'. Since all visibly identified erronous entries start with a lower case letter, this will be used to identify further erronous entries which will then be converted to 'None' in the Cleaning Process.

In [229]:
archive['Name_FirstLetter'] = archive['name'].str[0]
archive['Name_FirstLetter_Lower'] = archive['Name_FirstLetter'].str.islower()
archive_name_lower = archive.query('Name_FirstLetter_Lower==True')
archive_name_lower.name.value_counts()

a               55
the             8 
an              7 
very            5 
quite           4 
just            4 
one             4 
getting         2 
mad             2 
actually        2 
not             2 
space           1 
his             1 
all             1 
my              1 
old             1 
this            1 
infuriating     1 
by              1 
officially      1 
light           1 
unacceptable    1 
incredibly      1 
such            1 
life            1 
Name: name, dtype: int64

In [230]:
archive_name_none = archive_name_lower.name.value_counts().index.tolist()
archive_name_none

['a',
 'the',
 'an',
 'very',
 'quite',
 'just',
 'one',
 'getting',
 'mad',
 'actually',
 'not',
 'space',
 'his',
 'all',
 'my',
 'old',
 'this',
 'infuriating',
 'by',
 'officially',
 'light',
 'unacceptable',
 'incredibly',
 'such',
 'life']

In [231]:
# Check entries for columns **doggo, floofer, pupper, and puppo**
print(archive.pupper.value_counts())
print(archive.doggo.value_counts())
print(archive.puppo.value_counts())
print(archive.floofer.value_counts())
print("------------------------------------------")
print("Pupper: {}".format(archive.pupper.value_counts().loc['pupper']))
print("Doggo: {}".format(archive.doggo.value_counts().loc['doggo']))
print("Puppo: {}".format(archive.puppo.value_counts().loc['puppo']))
print("Floofer: {}".format(archive.floofer.value_counts().loc['floofer']))

None      2099
pupper    257 
Name: pupper, dtype: int64
None     2259
doggo    97  
Name: doggo, dtype: int64
None     2326
puppo    30  
Name: puppo, dtype: int64
None       2346
floofer    10  
Name: floofer, dtype: int64
------------------------------------------
Pupper: 257
Doggo: 97
Puppo: 30
Floofer: 10


Since out of the 2,356 available data records only 394 records have information on the type of dog this information will not be considered in the further analysis. If it were to be considered, one possible cleaning action would be to combine columns **doggo, floofer, pupper, and puppo** into one 'dogtype' column.

### Tweet Image Prediction

In [232]:
# Get a basic idea of columns in table and their values
image.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1645,808134635716833280,https://pbs.twimg.com/media/Cx5R8wPVEAALa9r.jpg,1,cocker_spaniel,0.740220,True,Dandie_Dinmont,0.061604,True,English_setter,0.041331,True
1847,839290600511926273,https://pbs.twimg.com/media/C6XBt9XXEAEEW9U.jpg,1,web_site,0.670892,False,monitor,0.101565,False,screen,0.075306,False
58,667090893657276420,https://pbs.twimg.com/media/CUH7oLuUsAELWib.jpg,1,Chihuahua,0.959514,True,Italian_greyhound,0.005370,True,Pomeranian,0.002641,True
171,668994913074286592,https://pbs.twimg.com/media/CUi_UtnWIAEtfqz.jpg,1,hog,0.113789,False,English_springer,0.089763,True,French_bulldog,0.082186,True
1735,821765923262631936,https://pbs.twimg.com/media/C2d_vnHWEAE9phX.jpg,1,golden_retriever,0.980071,True,Labrador_retriever,0.008758,True,Saluki,0.001806,True


In [233]:
# Get data types of columns and number of non-null entries
image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


The image prediction table has less entries than the Twitter archive table. Only a maximum of 2075 can be matched to 2356 tweets. No null values in this table.

In [234]:
#Get number of duplicated rows
sum(image.duplicated())

0

In [235]:
# Check for duplicated values in 'tweet_id'
sum(image.tweet_id.duplicated())   # could have also been analyzed by archive.tweet_id.nunique()  

0

There are no duplicated rows or tweet ids.

In [236]:
# Check the distribution on which image number corresponds to the most confident prediction
image.img_num.value_counts()

1    1780
2    198 
3    66  
4    31  
Name: img_num, dtype: int64

Most times the first image is the most confident prediction, but also p2, p3, and p4 have the best predictions. However information from p4 column is missing. Instead of deleting those rows from the data set the images with the average best fit are taken instead (lowest mean px_comf) (which is a bit simpler than identifying the second best fit for each row). 

In [237]:
image_p4 = image.query('img_num==4')
image_p4.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,3.100000e+01,31.0,31.000000,31.000000,31.000000
mean,8.002106e+17,4.0,0.818488,0.058523,0.027473
std,5.388178e+16,0.0,0.189390,0.053816,0.031909
min,6.686232e+17,4.0,0.226716,0.000023,0.000003
25%,7.606596e+17,4.0,0.771316,0.022395,0.007357
50%,8.186145e+17,4.0,0.883991,0.035059,0.017943
75%,8.339563e+17,4.0,0.944413,0.084091,0.029551
max,8.851676e+17,4.0,0.999953,0.204177,0.125745


If img_num is 4, p3 should be used instead (since it has lowest mean p3_conf value)

In [238]:
image.p2.value_counts().sort_index()

Afghan_hound                      5 
African_hunting_dog               1 
Airedale                          7 
American_Staffordshire_terrier    21
American_alligator                2 
American_black_bear               3 
Angora                            4 
Appenzeller                       3 
Arabian_camel                     4 
Arctic_fox                        4 
Australian_terrier                2 
Bedlington_terrier                5 
Bernese_mountain_dog              1 
Blenheim_spaniel                  4 
Border_collie                     12
Border_terrier                    4 
Boston_bull                       19
Brabancon_griffon                 10
Brittany_spaniel                  8 
Cardigan                          73
Chesapeake_Bay_retriever          41
Chihuahua                         44
Christmas_stocking                3 
Dandie_Dinmont                    4 
Doberman                          6 
English_foxhound                  2 
English_setter                    7 
E

When checking entries for p1, p2, and p3 it seems that real dog breeds are start with upper case letters whereas non dog breeds are start with lower case letters. This might be helpful later.

### Tweet Retweet Count

In [239]:
# Get a basic idea of columns in table and their values
#image.sample(5)

In [240]:
# Get data types of columns and number of non-null entries
#archive.info()

In [241]:
#Get number of duplicated rows
#sum(archive.duplicated())

In [242]:
# Check for duplicated values in 'tweet_id'
#sum(archive.tweet_id.duplicated())   # could have also been analyzed by archive.tweet_id.nunique()  

In [243]:
#image.describe()

### Quality Issues

**Twitter Archive**

- Column **timestamp** is not of type DateTime object
- Some values in column **rating_denominator** are not 10 (23 rows)
- Ratings higher or equal to 26 are either dur to wrong formatting or are outlier ratings in column **rating_numerator** (9 rows)
- Erronous entries in **name** column (stored in array archive_name_none)
- Not all columns are relevant for later analysis

**Tweet Image Prediction**

- Some dog breeds might be stored with a lower case first letter
- Column names are not informative
- Not all columns are relevant for later analysis

**Tweet Retweet Count**

#### Tidiness Issues

- Most confident prediction for dog (and breed) is stored in multiple columns (**Tweet Image Prediction** columns **p1, p2, p3, p4**); for img_num is 4, p3 should be used instead.

<a id='clean'></a>
## Clean Data

In [244]:
# Make copies of data frames
archive_clean = archive.copy()
image_clean = image.copy()
# retweet_clean = retweet.copy()

### Twitter Archive

#### Define
- Convert column **timestamp** to type DateTime object

#### Code

In [245]:
archive_clean['timestamp'] = pd.to_datetime(archive_clean['timestamp'])

#### Test

In [246]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
Name_FirstLetter        

Conversion to DateTime object successful.

#### Define
- Drop rows where **rating_denominator** is not 10 (23 rows)

#### Code

In [247]:
archive_clean = archive_clean[archive_clean.rating_denominator==10]

#### Test

In [248]:
archive_clean.shape[0]

2333

In [249]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2333 non-null int64
in_reply_to_status_id         73 non-null float64
in_reply_to_user_id           73 non-null float64
timestamp                     2333 non-null datetime64[ns]
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           180 non-null float64
retweeted_status_user_id      180 non-null float64
retweeted_status_timestamp    180 non-null object
expanded_urls                 2278 non-null object
rating_numerator              2333 non-null int64
rating_denominator            2333 non-null int64
name                          2333 non-null object
doggo                         2333 non-null object
floofer                       2333 non-null object
pupper                        2333 non-null object
puppo                         2333 non-null object
Name_FirstLetter        

Rows were deleted successfully (size before dropping was 2356, thus, 23 rows were dropped)

#### Define
- Keep only those rows where **rating_numerator** is lower than 26. Higher ratings are considered as wrong formatting or outliers (9 rows)

#### Code

In [250]:
archive_clean = archive_clean.query('rating_numerator<26')

#### Test

In [251]:
archive_clean.shape[0]

2324

Rows were deleted successfully (size before dropping was 2333, thus, 9 rows were dropped)

#### Define
- Create new column **dog_stage** for columns **doggo, floofer, pupper, and puppo**
note: melt function cannot be used since not all rows have defined dog stage and thus 'None' values cannot be removed afterwards (since those rows would then be removed as well)

#### Define
- Replcae erronous entries in **name** column with 'None' (stored in array archive_name_none)

#### Code

In [252]:
archive_clean.name.replace(archive_name_none, "None", inplace = True)

#### Test

In [253]:
print("{} 'None' in archive // {} 'None' in archive_clean".format(archive.name.value_counts().loc['None'], archive_clean.name.value_counts().loc['None']))

745 'None' in archive // 829 'None' in archive_clean


In [254]:
archive_name_lower.shape[0]

109

Count of "None" values for name was increased by 109, which is the number of entries in archive_name_lower (before 745, now 854)

### Tweet Image Prediction

#### Define
- Rename existing columns **px_dog** to 'isdog', **px** to'dogbreed', and **px_conf** to 'p_value' to store most confident prediction for dog and dog breed
- For img_num=1 use **p1**, for img_num=2 use **p2**, for img_num=3 use **p3**, and for img_num=4 use **p3**
- Capitalize dog breeds, since sometimes valid dog breeds are stored with lower case

#### Code

In [255]:
# Use p1_dog and p1 for img_num=1
image_clean_1 = image_clean
image_clean_1 = image_clean_1[image_clean_1.img_num==1]
image_clean_1 = image_clean_1.rename(columns = {'p1_dog':'isdog', 'p1':'dogbreed', 'p1_conf':'p_value'})

# Use p2_dog and p2 for img_num=2
image_clean_2 = image_clean
image_clean_2 = image_clean_2[image_clean_2.img_num==2]
image_clean_2 = image_clean_2.rename(columns = {'p2_dog':'isdog', 'p2':'dogbreed', 'p2_conf':'p_value'})

# Use p3_dog and p3 for img_num=3
image_clean_3 = image_clean
image_clean_3 = image_clean_3[image_clean_3.img_num==3]
image_clean_3 = image_clean_3.rename(columns = {'p3_dog':'isdog', 'p3':'dogbreed', 'p3_conf':'p_value'})

# Use p3_dog and p3 for img_num=4
image_clean_4 = image_clean
image_clean_4 = image_clean_4[image_clean_4.img_num==4]
image_clean_4 = image_clean_4.rename(columns = {'p3_dog':'isdog', 'p3':'dogbreed', 'p3_conf':'p_value'})

In [256]:
# Keept only columns 'tweetid, 'isdog', 'dogbreed', and 'p_value'
clean_1 = image_clean_1[['tweet_id','isdog','dogbreed','p_value']]
clean_2 = image_clean_2[['tweet_id','isdog','dogbreed','p_value']]
clean_3 = image_clean_3[['tweet_id','isdog','dogbreed','p_value']]
clean_4 = image_clean_4[['tweet_id','isdog','dogbreed','p_value']]

In [257]:
# Append data frames to one data frame
clean_1_2 = clean_1.append(clean_2, ignore_index=True)
clean_1_2_3 = clean_1_2.append(clean_3, ignore_index=True)
image_clean = clean_1_2_3.append(clean_4, ignore_index=True)

In [258]:
image_clean.dogbreed = image_clean.dogbreed.str.capitalize()

#### Test

In [259]:
image.shape[0]

2075

In [260]:
image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 4 columns):
tweet_id    2075 non-null int64
isdog       2075 non-null bool
dogbreed    2075 non-null object
p_value     2075 non-null float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 50.7+ KB


In [261]:
image_clean.dogbreed.value_counts()

Golden_retriever             123
Labrador_retriever           109
Pembroke                     80 
Chihuahua                    76 
Pug                          53 
Samoyed                      39 
Chow                         39 
Pomeranian                   38 
Toy_poodle                   37 
Cocker_spaniel               31 
Malamute                     30 
French_bulldog               27 
Eskimo_dog                   26 
Chesapeake_bay_retriever     23 
Kuvasz                       22 
Shetland_sheepdog            21 
Seat_belt                    21 
Staffordshire_bullterrier    21 
Cardigan                     21 
Siberian_husky               21 
Beagle                       19 
Italian_greyhound            19 
Pekinese                     19 
Miniature_pinscher           18 
Rottweiler                   16 
Kelpie                       16 
Lakeland_terrier             16 
Web_site                     16 
German_shepherd              16 
Teddy                        15 
          

The original data frame and the cleaned data frame have the same shape and the entries in column **dogbreed** are capitalized.

<a id='analysis'></a>
## Data Analysis

At least three (3) insights and one (1) visualization must be produced.

<a id='conc'></a>
## Conclusions